In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import ftplib
import sched, time

In [14]:
s = sched.scheduler(time.time, time.sleep)
def send_follower_matrix(sc): 
    
    # Read file
    
    session = ftplib.FTP('ftp.smartdiscussionplatform.com','file@smartdiscussionplatform.com','1YHDQ@i9XUl;')
    file = open('follower_matrix.csv','wb')                                             # name the file
    session.retrbinary('RETR public_html/matrixes/follower_matrix.csv', file.write)     # read the file
    file.close()                                                                        # close file and FTP
    session.quit()
    
    df = pd.read_csv('follower_matrix.csv')
    
    # Take user1-user2 nodes not user1-user1
    user_num = len(df)
    nodes = []
    for i in range(user_num):
        for j in range(user_num):
            if i==j:
                continue
            elif df.iloc[i][j] == 1:
                nodes.append([df.columns[i], df.columns[j]])    
                
                
    # Create a directed multi-graph
    G = nx.MultiDiGraph()
    G.add_edges_from(nodes)

    node_len = len(G)

    # colors
    colors = ['red','green','royalblue','yellow','cyan','orange','pink','red','chartreuse','magenta']
    color_len = len(colors)
    
    # Add colors to nodes
    node_colors = []
    if node_len < color_len:
        for color in range(node_len):
            node_colors.append(colors[color])
    else:
        color = 0
        total_color = 0
        while(True):
            if total_color == node_len:
                break
            elif color == color_len:
                color = 0
            node_colors.append(colors[color])
            color += 1
            total_color += 1
            
    
    # plot the graph
    plt.figure(figsize=(8,8))
    nx.draw(
        G, 
        node_size = 250, 
        with_labels = True,
        node_color = node_colors,
        arrowsize = 6,
        connectionstyle='arc3, rad = 0.5')

    # Save        
    plt.savefig("Follower_graph.png", format="PNG", dpi=300, bbox_inches='tight')

    #plt.show() 
    
    plt.close()
    
    session = ftplib.FTP('ftp.smartdiscussionplatform.com','file@smartdiscussionplatform.com','1YHDQ@i9XUl;')
    file = open('Follower_graph.png','rb')                                       # file to send
    session.storbinary('STOR public_html/matrixes/Follower_graph.png', file)     # send the file
    file.close()                                                                 # close file and FTP
    session.quit()
    
    # do your stuff
    s.enter(10, 1, send_follower_matrix, (sc,))

s.enter(10, 1, send_follower_matrix, (s,))
s.run()

KeyboardInterrupt: 